### **API SADVR - Portrait statistique**  
https://www.cen.umontreal.ca/espacedoc/sadvr/  

Le Service d’accès aux données de la Vitrine-Recherche (SADVR) est un service institutionnel de partage de données relatives aux profils des professeur·e·s et à leurs activités académiques ainsi qu'aux expertises et disciplines de recherche des facultés, départements et centres de recherche affiliés à l'Université de Montréal.  
  
Ce NoteBook est destiné à l'extraction et la visualisation de statistiques relatives aux professeur·e·s et à leurs expertises. 
Celles-ci seront intégrées en un tableau de bord ([PowerBI](https://wiki.umontreal.ca/display/SIE/Power+BI), [Dash](https://dash.plotly.com/) ou autre) permettant d'avoir un portrait d'ensemble des données.  

---

In [61]:
import pandas as pd
from SADVR_utils import *
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

**Import des données**

In [2]:
columns = pd.read_csv('columns.csv')['columns'].tolist()
fonctionsProf = pd.read_csv('fonctionsProfs.csv')['codeSad'].tolist()

# Mettre à jour les tables de données 
dataIndividus = updateInfoIndividus()
dataIndividus = dataIndividus[dataIndividus['nom'] != '?_?']
dataIndividus = dataIndividus[
    ['idsadvr', 'sexe', 'langues', 'institution', 'unitesRecherche', 'paysCode', 
     'paysNom', 'formations', 'prix', 'publication', 'communication']]

dataProfs = getAllProfsSOLR()
dataProfs = dataProfs[[x for x in dataProfs.columns if x in columns]]

data = dataProfs.merge(dataIndividus, on=['idsadvr'])

___
**Statistiques sociodémographiques**

- Genre
- Langues parlées
- Langues écrites
- Date d'obtention du dernier diplôme
- Institution de formation / Pays

In [3]:
demographics = data[['idsadvr', 'sexe', 'langues', 'formations', 'affiliations']]
toNormalize = ['langues', 'affiliations', 'formations', 'formations.disciplines', 'formations.institutions']
for c in toNormalize:
    demographics = explodeNormalize(demographics, c)

columns = pd.read_csv('columnsDemographics.csv')['columns'].tolist()
demographics = demographics[[x for x in demographics.columns if x in columns]]


demographics = demographics[demographics['affiliations.fonction.codeSad'].isin(fonctionsProf)]
demographics.to_csv('tables/demographics.csv', index=False)

nbProfs = len(demographics['idsadvr'].drop_duplicates())
print(f"Le répertoire de la vitrine de la recherche du SADVR compte actuellement {nbProfs} profils de professeur·e·s.")

Le répertoire de la vitrine de la recherche du SADVR compte actuellement 2609 profils de professeur·e·s.


In [4]:
demographics

,idsadvr,sexe,langues.nom,langues.medium,affiliations.fonction.codeSad,affiliations.fonction.nom,formations.diplome,formations.annee,formations.institutions.nom,formations.institutions.paysNom,formations.institutions.paysCode
0,in13580,M,NaN,NaN,42001,Professeur titulaire,NaN,NaN,NaN,NaN,NaN
1,in13580,M,NaN,NaN,42032,Professeur associé,NaN,NaN,NaN,NaN,NaN
3,in13581,M,Anglais,Écrit,42001,Professeur titulaire,doctorat IIIème cycle,1984,École Pratique des Hautes Études en Sciences S...,France,FR
4,in13581,M,Anglais,Écrit,42001,Professeur titulaire,doctorat IIIème cycle,1984,École Pratique des Hautes Études en Sciences S...,France,FR
6,in13581,M,Français,Écrit,42001,Professeur titulaire,doctorat IIIème cycle,1984,École Pratique des Hautes Études en Sciences S...,France,FR
...,...,...,...,...,...,...,...,...,...,...,...
11742,in15778,F,Français,Oral,42006,Professeure agrégée,Baccalauréat,2001,Univeristé d'Aix-Marseille II,France,FR
11743,in15778,F,Français,Oral,42006,Professeure agrégée,Maîtrise,2003,Université d'Aix-Marseille II,France,FR
11744,in15778,F,Français,Oral,42006,Professeure agrégée,Maîtrise,2003,Université d'Aix-Marseille II,France,FR
11745,in15778,F,Français,Oral,42006,Professeure agrégée,Doctorat (PhD),2010,Université de Montréal,Canada,CA


*Genre*

In [78]:
mapping = {'M': 'Hommes', 'F': 'Femmes', 'A': 'Autres'}
genre = plotVariable(demographics, 'sexe', mapping=mapping)

px.pie(
    names = genre['labels'],
    values = genre['count'],
    hole=0.5,
    title='Identité de genre',
    width=500,
    color_discrete_sequence= px.colors.qualitative.Antique   
)

*Langues (parlées, écrites)*

In [86]:
# Langues parlées
langueParle = demographics[demographics['langues.medium'] == 'Oral'].drop(columns=['langues.medium'])
langueParle = pd.DataFrame(plotVariable(langueParle, 'langues.nom'))
langueParle = groupOtherValues(langueParle)

px.pie(
    langueParle,
    values = langueParle['count'], 
    names = langueParle['labels'], 
    title='Langues parlées',
    hole=0.5,
    category_orders={'labels': 
        ['Français', 'Anglais', 'Espagnol; castillan', 
         'Allemand', 'Italien', 'Arabe', 'Autre']},
    color_discrete_sequence= px.colors.qualitative.Pastel1,  
    width=600
)

In [45]:
# Langues écrites

langueEcrite = demographics[demographics['langues.medium'] == 'Écrit'].drop(columns=['langues.medium'])
langueEcrite = pd.DataFrame(plotVariable(langueEcrite, 'langues.nom'))
langueEcrite = groupOtherValues(langueEcrite)

order = [x for x in langueEcrite['labels'] if not (x == 'Autre')] + ['Autre']

px.pie(
    langueEcrite,
    values = langueEcrite['count'], 
    names = langueEcrite['labels'], 
    title='Langues écrites',
    hole=0.5,
    category_orders={'labels': order},
    width=600
)

*Nombre de langues parlées*

In [8]:
langues = demographics[demographics['langues.medium'] == 'Oral'][['idsadvr', 'langues.nom', 'sexe']].drop_duplicates()
nbLangues = pd.DataFrame(langues.groupby(['idsadvr', 'sexe'])['langues.nom'].count()).reset_index().sort_values(by='langues.nom', ascending=False)
freqNbLangues = pd.DataFrame(nbLangues['langues.nom'].value_counts()).reset_index()
freqNbLangues

graphLangueEcrite = px.pie(
    freqNbLangues, 
    values='count', 
    names=freqNbLangues['langues.nom'].apply(lambda x: str(x) + " langue(s) parlée(s)"), 
    title='Nombre de langues parlées',
    hole=0.5,
    color = 'langues.nom',
    width=600
)

graphLangueEcrite.show()

*Nombre de langues parlées - Femmes vs Hommes*

In [9]:
# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

# Femmes
dataF = pd.DataFrame(nbLangues[nbLangues['sexe']=='F']['langues.nom'].value_counts()).reset_index()

# Hommes
dataH = pd.DataFrame(nbLangues[nbLangues['sexe']=='M']['langues.nom'].value_counts()).reset_index()

fig.add_trace(go.Pie(
    labels=dataF['langues.nom'].apply(lambda x: str(x) + ' langue(s) parlée(s)'), 
    values=dataF['count'], 
    title="Femmes"),
              1, 1)

fig.add_trace(go.Pie(
    labels=dataH['langues.nom'].apply(lambda x: str(x) + ' langue(s) parlée(s)'), 
    values=dataH['count'], 
    title="Hommes"),
              1, 2)


# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.5, hoverinfo="label+percent+value+name")

fig.update_layout(
    title_text="Nombre de langues parlées",
)

fig.to_html('./visualisations/nbLanguesParleesGenre.html')
fig.show()

*Fonction*

In [23]:
mappingFonction = pd.read_csv('tables/SADVR_fonctions.csv')[['codeSad', 'nomM']].to_dict('records')
mappingFonction = {x['codeSad'] : x['nomM'] for x in mappingFonction}

fonction = pd.DataFrame(plotVariable(demographics, 'affiliations.fonction.codeSad', mapping=mappingFonction))
fonction

,labels,count
0,Professeur titulaire,877
1,Professeur agrégé,429
2,Professeur associé,320
3,Professeur adjoint de clinique,269
4,Professeur adjoint,267
5,Professeur émérite,150
6,Professeur agrégé de clinique,144
7,Chercheur,78
8,Professeur accrédité,65
9,Professeur titulaire de clinique,42


In [24]:
fig = px.bar(
    fonction.sort_values(ascending=True, by='count'), 
    x="count", 
    y="labels", 
    orientation='h',
    title = 'Fonction',
    height=600)

fig.to_html('visualisations/fonctions.html')
fig.show()

In [25]:
fonctionGenre = demographics[['idsadvr', 'sexe', 'affiliations.fonction.codeSad']].drop_duplicates()
freqFonctionGenre = pd.DataFrame(fonctionGenre[['sexe', 'affiliations.fonction.codeSad']].value_counts()).reset_index()

mapping = pd.read_csv('tables/SADVR_fonctions.csv')[['codeSad', 'nomM']].to_dict('records')
mapping = {x['codeSad'] : x['nomM'] for x in mapping}

freqFonctionGenre['fonction'] = freqFonctionGenre['affiliations.fonction.codeSad'].map(mapping)
freqFonctionGenre.to_csv('tables/demographics/fonctionGenre.csv', index=False)

freqFonctionGenre = freqFonctionGenre[['sexe', 'fonction', 'count']]

fig = go.Figure()

# Femmes
dataF = freqFonctionGenre[freqFonctionGenre['sexe'] == 'F'].sort_values(by='count', ascending=True)

fig.add_trace(go.Bar(
    y= dataF['fonction'],
    x= dataF['count'],
    name='Femmes',
    marker_color='violet',
    orientation = 'h',
))

# Hommes
dataH = freqFonctionGenre[freqFonctionGenre['sexe'] == 'M'].sort_values(by='count', ascending=True)

fig.add_trace(go.Bar(
    y= dataH['fonction'],
    x= dataH['count'],
    name='Hommes',
    marker_color='lightblue',
    orientation = 'h'
))

# Autres
# dataA = freqFonctionGenre[freqFonctionGenre['sexe'] == 'A'].sort_values(by='count', ascending=True)

# fig.add_trace(go.Bar(
#     y= dataA['fonction'],
#     x= dataA['count'],
#     name='Autres',
#     marker_color='brown',
#     orientation = 'h'
# ))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(
    barmode='group', 
    xaxis_tickangle=-45,
    height=900,
    title = 'Distribution des fonctions par genre')

fig.show()

*Lieu de formation*

In [40]:
paysFormation = pd.DataFrame(plotVariable(demographics, 'formations.institutions.paysNom'))
paysFormation = groupOtherValues(paysFormation, 8)
paysFormation

,labels,count
0,Canada,488
1,États-Unis,175
2,France,110
3,Royaume-Uni,38
4,Suisse,15
5,Belgique,11
6,Pays-Bas,11
7,Allemagne,10
0,Autre,60


In [44]:
order = [x for x in paysFormation['labels'] if not (x == 'Autre')] + ['Autre']

px.pie(
    paysFormation, 
    values= paysFormation['count'], 
    names= paysFormation['labels'], 
    title='Formation universitaire Pays',
    hole=0.5,
    width=600, 
    category_orders= {'labels':
        order
    }
)

*Année d'obtention du dernier diplôme*

In [69]:
# D'abord filtrer pour ne conserver que le dernier diplôme obtenu
anneeDiplome = demographics.sort_values(['idsadvr', 'formations.annee'], ascending=[True, False])
anneeDiplome = anneeDiplome[['idsadvr', 'sexe', 'affiliations.fonction.nom', 'formations.annee']].dropna(subset='formations.annee')
anneeDiplome = anneeDiplome.drop_duplicates(subset=['idsadvr', 'formations.annee'])

print('N = ', len(anneeDiplome), f'({round((len(anneeDiplome) / nbProfs), 2) * 100} %)')
anneeDiplome = pd.DataFrame(plotVariable(anneeDiplome, 'formations.annee'))
anneeDiplome = anneeDiplome.sort_values(by='labels', ascending=True)
anneeDiplome

N =  1763 (68.0 %)


,labels,count
61,1959,1
56,1961,2
62,1962,1
60,1963,2
59,1964,2
...,...,...
43,2018,15
39,2019,17
37,2020,19
47,2021,8


In [74]:
fig = px.line(
    anneeDiplome, 
    x=anneeDiplome['labels'], 
    y=anneeDiplome['count'],
    title="Année d'obtention du dernier diplôme (Ph.D)")
fig.show()

**Expertises de recherche**

- Affiliations (facultés, départements, établissements affiliés)
- Départements
- Disciplines
- Secteur de recherche
- Périodes chronologiques étudiées
- Régions géographiques / pays étudiés

In [16]:
expertises = data[['idsadvr', 'affiliations', 'expertise']]

toNormalize = ['affiliations', 'expertise', 'expertise.secteursRecherche',
                'expertise.disciplines', 'expertise.pays', 
                'expertise.continents', 'expertise.periodesChronologiques']

for c in toNormalize:
    expertises = explodeNormalize(expertises, c)

drop = ['affiliations.courrielInstitutionnel', 'affiliations.immeuble',
        'affiliations.fonction.codeSad', 'affiliations.fonction.nom', 'expertise.motsCles',
       'affiliations.local', 'affiliations.exclusion', 'affiliations.exclusionTel', 'expertise.phraseCle',
       'affiliations.uniteAdministrative.codeSad', 'affiliations.uniteAdministrative.nom',
       'affiliations.telephone.numero', 'affiliations.telephone.poste']

expertises = expertises.drop(columns=drop)
expertises

,idsadvr,affiliations.departement.codeSad,affiliations.departement.nom,affiliations.faculte.codeSad,affiliations.faculte.nom,expertise.secteursRecherche.uid,expertise.secteursRecherche.codeLangue,expertise.secteursRecherche.nom,expertise.secteursRecherche.ordre,expertise.disciplines.uid,...,expertise.pays.codeLangue,expertise.pays.nom,expertise.continents.uid,expertise.continents.ordre,expertise.continents.codeLangue,expertise.continents.nom,expertise.periodesChronologiques.uid,expertise.periodesChronologiques.ordre,expertise.periodesChronologiques.codeLangue,expertise.periodesChronologiques.nom
0,in13580,2310,Département de biochimie et médecine moléculaire,23,Faculté de médecine,6,fre,Sciences de la santé,1,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,in13580,2310,Département de biochimie et médecine moléculaire,23,Faculté de médecine,6,fre,Sciences de la santé,1,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,in13580,2310,Département de biochimie et médecine moléculaire,23,Faculté de médecine,6,fre,Sciences de la santé,1,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,in13580,2310,Département de biochimie et médecine moléculaire,23,Faculté de médecine,6,fre,Sciences de la santé,1,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,in13580,2310,Département de biochimie et médecine moléculaire,23,Faculté de médecine,6,fre,Sciences de la santé,1,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284237,in19115,None,None,36,Faculté de pharmacie,7,eng,Medical Sciences,2,112,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284238,in19115,None,None,36,Faculté de pharmacie,7,eng,Medical Sciences,2,183,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284239,in19115,None,None,36,Faculté de pharmacie,7,eng,Medical Sciences,2,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284240,in19115,None,None,36,Faculté de pharmacie,7,eng,Medical Sciences,2,112,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Expertises de recherche: cartographie des expertises par mots-clés**

- Mots-clés / Phases clés associés aux disciplines / départements de recherche 

Voir si on peut extraire un graphe et le visualiser

**Rayonnement académique**

- Publications
- Communications
- Prix